In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

/Users/fl21879/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# define url
url = "https://www.twomlows.com"

In [3]:
# get HTML content
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

In [ ]:
# get desired HTML tags
selected_tags = ['address', 'title', 'header', 'footer']
for tag_name in selected_tags:
    found_tags = soup.find_all(tag_name)
    
    # print tag name and content
    if found_tags:
        for tag in found_tags:
            tag_content = tag.text.strip()
            print(f"Content of <{tag_name}>:")
            if tag_content:
                print(tag_content)
                print("-"*40) # separator line
            else:
                print(f"<{tag_name}> is empty")
                print("-"*40)
    else:
        print(f"<{tag_name}> not found")
        print("-"*40)

<address> not found
----------------------------------------
Content of <title>:
Twomlows Solicitors - Twomlows Monmouth Solicitors
----------------------------------------
Content of <header>:
enquiries@twomlows.com















 
Caldicot















 
Chepstow















 
Monmouth














Facebook-f
 



Linkedin
 















 









Practice Areas

Accident Injury Claims
Commercial Business Advice
Criminal Defence Specialists
Dispute Resolution
Family Matrimonial Solicitor
Lasting Powers of Attorney
Probate Administration Estates
Residential Conveyancing
Will Writing


Recruitment
The Team
Offices

Monmouth Solicitors
Chepstow Solicitors
Caldicot Solicitors


 

 

Practice Areas

Accident Injury Claims
Commercial Business Advice
Criminal Defence Specialists
Dispute Resolution
Family Matrimonial Solicitor
Lasting Powers of Attorney
Probate Administration Estates
Residential Conveyancing
Will Writing


Recruitment
The Team
Offices

Monmouth Solicitors
Chepstow